In [1]:
import numpy as np
import gym
import tensorflow as tf

In [2]:
env = gym.make("CartPole-v0")

[2017-01-14 07:44:19,262] Making new env: CartPole-v0


In [3]:
H = 10
batch_size = 5
learning_rate = 1e-2
gamma = 0.99

D = 4

In [4]:
tf.reset_default_graph()
var_init = tf.contrib.layers.xavier_initializer()

observations = tf.placeholder(tf.float32, [None, D], name="input_x")

w1 = tf.get_variable("w1", shape=[D,H], initializer=var_init)
layer1 = tf.nn.relu(tf.matmul(observations, w1))

w2 = tf.get_variable("w2", shape=[H,1], initializer=var_init)
layer2 = tf.nn.relu(tf.matmul(layer1, w2))

probability = tf.nn.sigmoid(layer2)

In [5]:
tvars = tf.trainable_variables()
input_y = tf.placeholder(tf.float32, [None,1], name="input_y")
advantages = tf.placeholder(tf.float32, name="reward_signal")

In [6]:
loglik = tf.log(input_y*(input_y - probability) + (1 - input_y)*(input_y + probability))
loss = -tf.reduce_mean(loglik * advantages)
newGrads = tf.gradients(loss, tvars)

In [7]:
adam = tf.train.AdamOptimizer(learning_rate = learning_rate)

W1Grad = tf.placeholder(tf.float32, name="batch_grad1")
W2Grad = tf.placeholder(tf.float32, name="batch_grad2")

batchGrad = [W1Grad, W2Grad]
updateGrads = adam.apply_gradients(zip(batchGrad, tvars))

In [11]:
def discount_rewards(r):
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(range(0, r.size)):
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r[t]

In [ ]:
xs, ys, drs = [],[],[]
running_reward = None
reward_sum = 0
episode_number = 1
total_episodes = 10000
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    observation = env.reset()
    
    gradBuffer = sess.run(tvars)
    for ix, grad in enumerate(gradBuffer):
        gradBuffer[ix] = grad * 0
        
    while episode_number <= total_episodes:
        x = np.reshape(observation, [1,D])
        
        tfprob = sess.run(probability, feed_dict={observations: x})
        action = 1 if np.random.uniform() < tfprob else 0
        
        xs.append(x)
        y = 1 if action == 0 else 0
        ys.append(y)
        
        observation, reward, done, _ = env.step(action)
        reward_sum += reward
        
        drs.append(reward)
        
        if done:
            episode_number += 1
            
            epx = np.vstack(xs)
            epy = np.vstack(ys)
            epr = np.vstack(drs)
            
            discounted_epr = discount_rewards(epr)
            discounted_epr -= np.mean(discounted_epr)
            discounted_epr /= np.std(discounted_epr)
            
            tGrad = sess.run(newGrads,feed_dict={observations:epx, input_y:epy, advantages: discounted_epr})
            for ix, grad in enumerate(tGrad):
                gradBuffer[ix] += grad
            
            if episode_number % batch_size == 0:
                sess.run(updateGrads, feed_dict={W1Grad: gradBuffer[0],W2Grad:gradBuffer[1]})
                for ix,grad in enumerate(gradBuffer):
                    gradBuffer[ix] = grad * 0
            
                running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
                print("average reward for episode %f Total average reward %f"%(reward_sum/batch_size, running_reward/batch_size))
            
                reward_sum = 0
        
            observation = env.reset()
        
        
        

average reward for episode 18.000000 Total average reward 18.000000
average reward for episode 9.400000 Total average reward 17.914000
average reward for episode 9.400000 Total average reward 17.828860
average reward for episode 9.200000 Total average reward 17.742571
average reward for episode 9.200000 Total average reward 17.657146
average reward for episode 9.800000 Total average reward 17.578574
average reward for episode 9.600000 Total average reward 17.498788
average reward for episode 9.600000 Total average reward 17.419801
average reward for episode 10.000000 Total average reward 17.345603
average reward for episode 10.000000 Total average reward 17.272147
average reward for episode 9.200000 Total average reward 17.191425
average reward for episode 10.200000 Total average reward 17.121511
average reward for episode 9.600000 Total average reward 17.046296
average reward for episode 9.400000 Total average reward 16.969833
average reward for episode 9.600000 Total average reward 1